In [ ]:
import os
import pandas as pd
import pickle
from ast import literal_eval
import logging
import argparse
from simpletransformers.ner import (
    NERArgs,
    NERModel,
)

In [ ]:
BIO_type = 1

labelset = ["B-p", "I-p", "B-n", "I-n", "O"]
type_ls = ['-p', '-n']

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

pos = pd.read_csv('pos_sent.csv')
col_name = pos.columns[6+BIO_type]
pos = pos.dropna(axis=0, subset=[col_name])

In [ ]:
data = []
for i in range(len(pos)):
    words = pos.loc[i, 'text'].split(' ')
    tags = literal_eval(pos.iloc[i, 6+BIO_type])
    for j in range(len(words)):
        data.append([i, words[j], tags[j]])
df = pd.DataFrame(data, columns=['sentence_id', 'words', 'labels'])

In [ ]:
model_args = NERArgs()
model_args.reprocess_input_data = True
model_args.save_model_every_epoch = False
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.manual_seed = 1
model_args.use_multiprocessing = True
model_args.do_lower_case = False

In [ ]:
def phrase_F1(ref, pred):
    return 0

# Create a TransformerModel
model = NERModel(
    "xlmroberta",
    "../ner/Noutputs/best_model",
    labels=labelset,
    args=model_args,
)

In [ ]:
result, model_outputs, pred_label = model.eval_model(df, F1_score=phrase_F1)
# 'pred_label' is a list, where each element is the list of predicted labels for one sentence

In [ ]:
p=[str(l) for l in pred_label]

In [ ]:
pos = pd.read_csv('pos_sent.csv')
pos['BIO_1']=p

In [ ]:
pos

In [ ]:
pos.to_csv('pos_sent.csv')

In [ ]:
def get_entity_spans(k, ls):
    spans = [[],[]]
    for i in range(len(ls)):
        if ls[i]=='B-p':
            for j in range(i+1,len(ls)):
                if ls[j]!='I-p':
                    phrase=' '.join(pos.loc[k,'text'].split(' ')[i:j])
                    tup=(phrase, (i, j))
                    spans[0].append(tup)
                    break
                elif j==len(ls)-1:
                    phrase=' '.join(pos.loc[k,'text'].split(' ')[i:(j+1)])
                    tup=(phrase, (i, (j+1)))
                    spans[0].append(tup)
        elif ls[i]=='B-n':
            for j in range(i+1,len(ls)):
                if ls[j]!='I-n':
                    phrase=' '.join(pos.loc[k,'text'].split(' ')[i:j])
                    tup=(phrase, (i, j))
                    spans[1].append(tup)
                    break
                elif j==len(ls)-1:
                    phrase=' '.join(pos.loc[k,'text'].split(' ')[i:(j+1)])
                    tup=(phrase, (i, (j+1)))
                    spans[1].append(tup)
    return spans
                    

In [ ]:
all=[]
for k in range(len(pred_label)):
    all.append(get_entity_spans(k, pred_label[k]))
all

In [ ]:
all[35:40]

In [ ]:
dt=pd.read_csv('pos_sent.csv')
#dt=dt['labels','text']
a=pd.DataFrame(all)
a.columns=['predicates', 'subj/obj']
a=pd.concat([dt[['labels','text']],a],axis=1)
a['triple_A']='[]'
a['triple_B']='[]'
a['triple_C']='[]'
a['triple_D']='[]'
a=a.reset_index(drop=True)

In [ ]:
a.to_csv('triples.csv',index=False)